In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
import random 

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Estadísticos
# ------------------------------------------------------------------------------
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

/home/cassia_d/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/cassia_d/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# Pair Programming ANOVA

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

In [25]:
df = pd.read_csv('datos/soci_econ_country_profiles_v5.csv', index_col=0)
df.head(2)

,country,Region,GDP per capita (current US$),Employment: Agriculture (% of employed),Employment: Services (% of employed),Agricultural production index (2004-2006=100),Urban population (% of total population)_x,Urban population growth rate (average annual %),"Fertility rate, total (live births per woman)",Infant mortality rate (per 1000 live births,...,Human capital index (HCI) (scale 0-1),"Inflation, consumer prices (annual %)","Life expectancy at birth, total (years)",Labour force participation (male pop. %),"Pop. using improved drinking water (urban, %)","Pop. using improved drinking water (rural, %)","Population age distribution (0-14, %)","Population age distribution (60+ years, %)",Education: Secondary gross enrol. ratio (per 100 pop.),Education: Tertiary gross enrol. ratio (per 100 pop.)
0,Argentina,SouthAmerica,-0.446268,-0.779156,0.604781,0.453905,1.136844,-0.141090,0.832595,0.403148,...,-0.691809,NaN,-0.376146,0.534838,0.220883,0.590834,0.998345,-0.493168,0.128973,0.934014
1,Australia,Oceania,1.346217,-0.715406,0.840977,0.020821,1.001917,0.279591,0.111013,-0.501415,...,1.022784,-0.267149,1.033843,0.058083,0.681103,0.695233,-0.023719,0.248316,1.683651,1.274595


In [26]:
df.rename(columns = {col : col.replace(' ', '_') for col in df.columns}, inplace=True)

In [27]:
diccionario = {"country": "country",
                "Region": "Region",
                "GDP_per_capita_(current_US$)" : "GDP_per_capita",
                "Employment:_Agriculture_(%_of_employed)" : "Employment_Agriculture",
                "Employment:_Services_(%_of_employed)" : "Employment_Services",
                "Agricultural_production_index_(2004-2006=100)" : "Agricultural_production_index",
                "Urban_population_(%_of_total_population)_x" : "Urban_population",
                "Urban_population_growth_rate_(average_annual_%)" : "Urban_population_growth_rate",
                "Fertility_rate,_total_(live_births_per_woman)" : "Fertility_rate_total",
                "Infant_mortality_rate_(per_1000_live_births" : "Infant_mortality_rate",
                "Seats_held_by_women_in_national_parliaments_%" : "Seats_held_by_women_in_national_parliaments",
                "Individuals_using_the_Internet_(per_100_inhabitants)" : "Individuals_using_the_Internet",
                "Energy_production,_primary_(Petajoules)" : "Energy_production_primary",
                "Energy_supply_per_capita_(Gigajoules)" : "Energy_supply_per_capita",
                "Quality_Of_Life_Index" : "Quality_Of_Life_Index",
                "Purchasing_Power_Index" : "Purchasing_Power_Index",
                "Safety_Index" : "Safety_Index",
                "Health_Care_Index" : "Health_Care_Index",
                "Property_price_to_income_ratio" : "Property_price_to_income_ratio",
                "Traffic_commute_time_index" : "Traffic_commute_time_index",
                "Pollution_index" : "Pollution_index",
                "Climate_index" : "Climate_index",
                "Affordability_Index" : "Affordability_Index",
                "Cost_Of_Living_Index" : "Cost_Of_Living_Index",
                "Consumer_price_index_(2010_=_100)" : "Consumer_price_index",
                "Current_health_expenditure_(%_of_GDP)" : "Current_health_expenditure",
                "Human_capital_index_(HCI)_(scale_0-1)" : "Human_capital_index",
                "Inflation,_consumer_prices_(annual_%)" : "Inflation_consumer_prices",
                "Life_expectancy_at_birth,_total_(years)" : "Life_expectancy_at_birth_total",
                "Labour_force_participation_(male_pop._%)" : "Labour_force_participation",
                "Pop._using_improved_drinking_water_(urban,_%)" : "Pop_using_improved_drinking_water_urban",
                "Pop._using_improved_drinking_water_(rural,_%)" : "Pop_using_improved_drinking_water_rural",
                "Population_age_distribution_(0-14,_%)" : "Population_age_distribution_0_14",
                "Population_age_distribution_(60+_years,_%)" : "Population_age_distribution_60_mas",
                "Education:_Secondary_gross_enrol._ratio_(per_100_pop.)" : "Education_Secondary_gross_enrol_ratio",
                "Education:_Tertiary_gross_enrol._ratio_(per_100_pop.)" : "Education_Tertiary_gross_enrol_ratio"}


In [28]:
df.rename(columns = diccionario, inplace=True)

In [20]:
df.head(2)

,country,Region,GDP_per_capita,Employment_Agriculture,Employment_Services,Agricultural_production_index,Urban_population,Urban_population_growth_rate,"Fertility_rate,_total",Infant_mortality_rate,...,Human_capital_index,Inflation_consumer_prices,Life_expectancy_at_birth_total,Labour_force_participation,Pop_using_improved_drinking_water_urban,Pop_using_improved_drinking_water_rural,Population_age_distribution_0_14,Population_age_distribution_60_mas,Education_Secondary_gross_enrol_ratio,Education_Tertiary_gross_enrol_ratio
0,Argentina,SouthAmerica,-0.446268,-0.779156,0.604781,0.453905,1.136844,-0.141090,0.832595,0.403148,...,-0.691809,NaN,-0.376146,0.534838,0.220883,0.590834,0.998345,-0.493168,0.128973,0.934014
1,Australia,Oceania,1.346217,-0.715406,0.840977,0.020821,1.001917,0.279591,0.111013,-0.501415,...,1.022784,-0.267149,1.033843,0.058083,0.681103,0.695233,-0.023719,0.248316,1.683651,1.274595


In [21]:
for col in df.columns:
    print(f'+ {col}')

+ country
+ Region
+ GDP_per_capita
+ Employment_Agriculture
+ Employment_Services
+ Agricultural_production_index
+ Urban_population
+ Urban_population_growth_rate
+ Fertility_rate,_total
+ Infant_mortality_rate
+ Seats_held_by_women_in_national_parliaments
+ Individuals_using_the_Internet
+ Energy_production_primary
+ Energy_supply_per_capita
+ Quality_Of_Life_Index
+ Purchasing_Power_Index
+ Safety_Index
+ Health_Care_Index
+ Property_price_to_income_ratio
+ Traffic_commute_time_index
+ Pollution_index
+ Climate_index
+ Affordability_Index
+ Cost_Of_Living_Index
+ Consumer_price_index
+ Current_health_expenditure
+ Human_capital_index
+ Inflation_consumer_prices
+ Life_expectancy_at_birth_total
+ Labour_force_participation
+ Pop_using_improved_drinking_water_urban
+ Pop_using_improved_drinking_water_rural
+ Population_age_distribution_0_14
+ Population_age_distribution_60_mas
+ Education_Secondary_gross_enrol_ratio
+ Education_Tertiary_gross_enrol_ratio


In [29]:
lm = ols('''Quality_Of_Life_Index ~ country
            + Region
            + GDP_per_capita
            + Employment_Agriculture
            + Employment_Services
            + Agricultural_production_index
            + Urban_population
            + Urban_population_growth_rate
            + Fertility_rate_total
            + Infant_mortality_rate
            + Seats_held_by_women_in_national_parliaments
            + Individuals_using_the_Internet
            + Energy_production_primary
            + Energy_supply_per_capita
            + Purchasing_Power_Index
            + Safety_Index
            + Health_Care_Index
            + Property_price_to_income_ratio
            + Traffic_commute_time_index
            + Pollution_index
            + Climate_index
            + Affordability_Index
            + Cost_Of_Living_Index
            + Consumer_price_index
            + Current_health_expenditure
            + Human_capital_index
            + Inflation_consumer_prices
            + Life_expectancy_at_birth_total
            + Labour_force_participation
            + Pop_using_improved_drinking_water_urban
            + Pop_using_improved_drinking_water_rural
            + Population_age_distribution_0_14
            + Population_age_distribution_60_mas
            + Education_Secondary_gross_enrol_ratio
            + Education_Tertiary_gross_enrol_ratio''', data=df).fit()
sm.stats.anova_lm(lm)

ValueError: shapes (36,111) and (56,) not aligned: 111 (dim 1) != 56 (dim 0)